In [ ]:
from itertools import product, combinations, count, permutations, starmap, chain, repeat
import numpy as np
import pandas as pd
from astroquery.vizier import Vizier
import matplotlib.pyplot as plt
import astropy as ap
import sklearn
from sklearn.neighbors import KernelDensity as KD
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA, KernelPCA
import corner
from sklearn.mixture import GaussianMixture as GM
from sklearn.mixture import BayesianGaussianMixture as BGM
from scipy.stats import norm
from matplotlib.colors import LogNorm
from astroquery.gaia import Gaia
from scipy import stats

In [ ]:
%matplotlib inline

In [ ]:
Vizier = Vizier(row_limit=20000)

In [ ]:
catalog = Vizier.get_catalogs("J/A+A/618/A93")

clucata = catalog[1]

In [ ]:
newc = clucata.group_by('Cluster')
maxcluster = np.argmax(newc.groups.indices[1:]-newc.groups.indices[:-1])
clusterid = maxcluster+6
bigcluster = newc.groups[clusterid]
print(bigcluster.colnames)
newc.groups[clusterid]['Cluster'][0]

Good `'Cluster'`s to choose from: Alessi\_24, ASCC_99, Alessi\_12

In [ ]:
cutcluster = bigcluster[bigcluster['PMemb']>.8]
cutcluster = cutcluster[~np.isnan(cutcluster["BP-RP"])]
#We should cut by lines away from main sequence
cutcluster = cutcluster[~np.logical_and(cutcluster["BP-RP"]>1.0,cutcluster['Gmag']<10.)]
cutcluster = cutcluster[~np.logical_and(cutcluster["BP-RP"]<.7, cutcluster['Gmag']>13.8)]
plt.plot(cutcluster['RA_ICRS'],cutcluster['DE_ICRS'],'+')
plt.title('angular coordinates of '+cutcluster['Cluster'][0])

In [ ]:
query = 'SELECT source_id, phot_bp_mean_mag, phot_rp_mean_mag FROM gaiadr2.gaia_source WHERE source_id = {}'

In [ ]:
newquery = query.format(' OR source_id = '.join(list(map(str, cutcluster['Source']))))
newquery;

In [ ]:
Gaia.launch_job('SELECT source_id, phot_bp_mean_mag, phot_rp_mean_mag FROM gaiadr2.gaia_source WHERE source_id = 1842846157478382720')

In [ ]:
newjob = Gaia.launch_job(query=newquery)

In [ ]:
results = newjob.get_results()

In [ ]:
results['phot_bp_mean_mag'].info.parent_table;

In [ ]:
cutcluster.add_columns([results['phot_bp_mean_mag'],results['phot_rp_mean_mag']])

In [ ]:
the_color = 'BP-RP'
the_mag = 'phot_rp_mean_mag'

In [ ]:
plt.hist(cutcluster['PMemb'])
plt.xlabel('PMemb')
plt.title('cluster membership probability of '+cutcluster['Cluster'][0])

In [ ]:
plt.plot(cutcluster[the_color],cutcluster[the_mag], '+')
plt.ylim(19, 7)
plt.xlabel('color')
plt.ylabel('magnitude')
plt.title('color-magnitude diagram of '+cutcluster['Cluster'][0])

PCA - diagram, inverse of variance as metric -> sqrt(thing)= transformation, apply ->PCA(test) -> KDE -> inverse transform both L and R on Kernel Widths

LLE? -> Local linear embedding -> for non MS

remove outliers before PCA

In [ ]:
X = np.asarray(np.vstack((cutcluster[the_color], cutcluster[the_mag]))).T

In [ ]:
pca = PCA(n_components=2)
X_pca = pca.fit(X)

In [ ]:
X_transform = pca.transform(X)
X_transform.shape
plt.scatter(X_transform[:,0], X_transform[:,1])
plt.axis("equal")

In [ ]:
np.sqrt(pca.singular_values_)

In [ ]:
def draw_vector(v0, v1, ax=None):
    ax = ax or plt.gca()
    arrowprops=dict(arrowstyle='->',
                    linewidth=2,
                    shrinkA=0, shrinkB=0)
    ax.annotate('', v1, v0, arrowprops=arrowprops)

# plot data
plt.scatter(X[:, 0], X[:, 1], alpha=0.2)
for length, vector in zip(pca.explained_variance_, pca.components_):
    v = vector * 3 * np.sqrt(length)
    draw_vector(pca.mean_, pca.mean_ + v)
plt.axis('equal');
plt.ylim(18,7)
plt.xlim(-2,4)

In [ ]:
X_pca = pca.transform(X)
#plt.scatter(X_pca[:, 0], X_pca[:, 1])
plt.axis('equal')
X_pca[:,0] = X_pca[:,0]/np.sqrt(pca.singular_values_[0])
X_pca[:,1] = X_pca[:,1]/np.sqrt(pca.singular_values_[1])
plt.scatter(X_pca[:, 0], X_pca[:, 1]);

In [ ]:
# I couldn't immediately find a KDE code that enabled different bandwidth in each dimension
# which we want because the errors in color are much greater than the errors in magnitude
params = {'bandwidth': np.logspace(-2, 2, 200)}
grid = GridSearchCV(KD(kernel='gaussian'), params, cv=10)
grid.fit(X_pca)

print("best bandwidth: {0}".format(grid.best_estimator_.bandwidth))
# first attempt obviously too fine a bandwidth because it allows for double stars
# we could fix it here or say this is just what the data is and fit an HRD model that doesn't permit those
# so now the data is the KDE evaluated on a grid

xmin = -1
xmax = +2
ymin = -1
ymax = +1
xlen = 50
ylen = 50


kde = grid.best_estimator_.fit(X_pca)
eval_where = np.array(list(product(np.linspace(xmin,xmax,xlen), np.linspace(ymin,ymax, ylen))))
log_dens = kde.score_samples(eval_where)

plt.imshow(np.flip(np.exp(log_dens.reshape(xlen, ylen).T), axis=0),
           extent=[xmin, xmax, ymin, ymax])
#plt.scatter(cutcluster['BP-RP'], cutcluster[the_mag], marker='.', color='r', s=1)

In [ ]:
a = np.linspace(xmin,xmax,xlen)*np.sqrt(pca.singular_values_[0])
b = np.linspace(ymin,ymax,ylen)*np.sqrt(pca.singular_values_[1])
A, B = np.meshgrid(a, b)
print(B.shape)
plt.contour(A, B, np.exp(log_dens.reshape(xlen,ylen).T));

In [ ]:
C_grid = np.array([np.ravel(A), np.ravel(B)]).T
E_grid = pca.inverse_transform(C_grid)[:,0].reshape(xlen, ylen)
F_grid = pca.inverse_transform(C_grid)[:,1].reshape(xlen, ylen)

In [ ]:
plt.plot(cutcluster[the_color],cutcluster[the_mag], 'r+',alpha=.4)
plt.contour(E_grid, F_grid, np.exp(log_dens.reshape(xlen,ylen).T))
plt.ylim(19,8)
plt.xlabel(the_color)
plt.ylabel(the_mag)
plt.title('color-magnitude diagram of '+cutcluster['Cluster'][0])
plt.savefig("cmd.png")

In [ ]:
samps = np.array([[x*np.sqrt(pca.singular_values_[0]),y*np.sqrt(pca.singular_values_[1])] for x,y in kde.sample(100)])

In [ ]:
cmsamps = pca.inverse_transform(samps)

In [ ]:
mags = cmsamps[:,1]
colors = cmsamps[:,0]

In [ ]:
toflux = lambda m: 10.**(-2/5.*m)
tomag = lambda ϕ: -5./2*np.log10(ϕ)

In [ ]:
tomag(toflux(10))

In [ ]:
magsum = lambda s: tomag(np.sum(list(map(toflux,s[0]))))
colorsum = lambda s: tomag(np.sum(list(map(toflux,s[0]+s[1]))))-magsum(s)

In [ ]:
makepairwise = lambda x,y: np.swapaxes(np.array(list(combinations(zip(x,y), 2))), -1, -2)
# makepairwise_helper = lambda x,y: np.array(np.meshgrid(x,y)).reshape(2, len(x)*len(y)).T
# makepairwise = lambda x,y: np.hstack((makepairwise_helper(x, y), makepairwise_helper(x, y)))
#makepairwise = lambda x,y: np.meshgrid(((x,y), (x,y)))

In [ ]:
newmags = np.array(list(map(magsum, makepairwise(mags,colors))))
newcolors = np.array(list(map(colorsum, makepairwise(mags,colors))))

In [ ]:
plt.scatter(newcolors, newmags)

### END OF REAL WORK